In [ ]:
import os
import glob
import numpy as np
from tensorflow.python.keras import layers
from tensorflow import keras 
import tensorflow as tf


data = os.listdir('data5')

print(len(data))

In [ ]:
def load_data(root, vfold_ratio=0.2, max_items_per_class= 6000 ):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    #initialize variables 
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []

    #load each data file 
    for idx, file in enumerate(all_files):
        data = np.load(file)
        if data.shape[0] < max_items_per_class:
            continue

        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)
        
        
    data = None
    labels = None
    
    #randomize the dataset 
    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]

    #separate into training and testing 
    vfold_size = int(x.shape[0]/100*(vfold_ratio*100))

    x_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    x_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]
    return x_train, y_train, x_test, y_test, class_names

In [ ]:
x_train, y_train, x_test, y_test, class_names = load_data('data5')
num_classes = len(class_names)
image_size = 28
print(class_names)

In [ ]:
print("Train: ",len(x_train))
print("Test: ",len(x_test))

import matplotlib.pyplot as plt
from random import randint
%matplotlib inline  
idx = randint(0, len(x_train))
plt.imshow(x_train[idx].reshape(28,28)) 
print(int(y_train[idx]))

In [ ]:

x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')
print(x_train.shape[0])
x_train /= 255.0
x_test /= 255.0


# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

print(y_train.shape)
print(num_classes)
print(len(x_train))



In [ ]:


from PIL import Image
import numpy as np
from ultralytics import YOLO
import tensorflow as tf


model = YOLO(r'YoloComposedColab100\weights\best.pt')

imgs = []
imgsTest = []
results = []
resultsTest = []
for i in range(0,10000):
    img = Image.fromarray(np.uint8(x_train[i].reshape((28, 28))*255))
    imgs.append(img.resize((256, 256)))
    results.append(model.predict(source=img.resize((256, 256)), save=False,  conf=0.25, task='detect')[0])



In [ ]:
for i in range(10000,20000):
    img = Image.fromarray(np.uint8(x_train[i].reshape((28, 28))*255))
    imgs.append(img.resize((256, 256)))
    results.append(model.predict(source=img.resize((256, 256)), save=False,  conf=0.25, task='detect')[0])

In [ ]:
for i in range(20000,24000):
    img = Image.fromarray(np.uint8(x_train[i].reshape((28, 28))*255))
    imgs.append(img.resize((256, 256)))
    results.append(model.predict(source=img.resize((256, 256)), save=False,  conf=0.25, task='detect')[0])

In [ ]:
for i in range(0,len(y_test)):
    img = Image.fromarray(np.uint8(x_train[i].reshape((28, 28))*255))
    imgsTest.append(img.resize((256, 256)))
    resultsTest.append(model.predict(source=img.resize((256, 256)), save=False,  conf=0.25, task='detect')[0])    

In [ ]:
print(" results length: ",len(results), "\n", "imgs length: ", len(imgs))   
print(" resultsTest length: ",len(resultsTest), "\n", "imgsTest length: ", len(imgsTest))   

In [ ]:
import cv2
from PIL import Image
import numpy as np

train_data = []
    # Set the threshold for object detection
conf_threshold = 0.25

    # Set the threshold for non-maximum suppression
nms_threshold = 0.4


for k in range(0,len(imgs)):
    # Create a list to store the bounding boxes and class names for the detected objects
    boxes = []
    class_names2 = []
    img = imgs[k]
    # Create a new image with the same dimensions as the original image
    new_img = np.zeros_like(img)


    # Get the bounding boxes and scores
    scores = []
  

    for i in range(len(results[k].boxes.cls)):
        scores.append(float(results[k].boxes.cls[i]))

    for j in range(len(results[k].boxes.xywh)):
        center_x = int(results[k].boxes.xywh[j][0])
        center_y = int(results[k].boxes.xywh[j][1])
        w = int(results[k].boxes.xywh[j][2])
        h = int(results[k].boxes.xywh[j][3])
        x = int(center_x - w/2)
        y = int(center_y - h/2)
        box = [x, y, w, h]
        boxes.append(box)
        class_names2.append(scores[j])

    new_img = Image.fromarray(new_img)
    # Loop over the selected boxes
    for i in range(0,len(boxes)):
        box = boxes[i]
        class_name = class_names2[i]
        
        # Crop the object from the image
        crop_img = img.crop((int(box[0]), int(box[1]), int(box[0])+int(box[2]), int(box[1])+int(box[3])))
        
        # Paste the cropped object onto the new image
        new_img.paste(crop_img, (int(box[0]), int(box[1]), int(box[0])+int(box[2]), int(box[1])+int(box[3])))

    # Save the new image
    if len(results[k].boxes) == 0:
        train_data.append(np.asarray(imgs[k]))
    else:    
        train_data.append(np.asarray(new_img))


print(len(train_data))    


In [ ]:
import cv2
import numpy as np

test_data = []
    # Set the threshold for object detection
conf_threshold = 0.25

    # Set the threshold for non-maximum suppression
nms_threshold = 0.4


for k in range(0,len(imgsTest)):
    # Create a list to store the bounding boxes and class names for the detected objects
    boxes = []
    class_names2 = []
    # img = cv2.imread('untitled.png')
    img = imgsTest[k]
    # Create a new image with the same dimensions as the original image
    new_img = np.zeros_like(img)


    # Get the bounding boxes and scores
    scores = []
    for i in range(len(resultsTest[k].boxes.cls)):
        scores.append(float(resultsTest[k].boxes.cls[i]))

    for j in range(len(resultsTest[k].boxes.xywh)):
        center_x = int(resultsTest[k].boxes.xywh[j][0])
        center_y = int(resultsTest[k].boxes.xywh[j][1])
        w = int(resultsTest[k].boxes.xywh[j][2])
        h = int(resultsTest[k].boxes.xywh[j][3])
        x = int(center_x - w/2)
        y = int(center_y - h/2)
        box = [x, y, w, h]
        boxes.append(box)
        class_names2.append(scores[j])

    # Apply non-maximum suppression
    new_img = Image.fromarray(new_img)
    # Loop over the selected boxes
    for i in range(0,len(boxes)):
        box = boxes[i]
        class_name = class_names2[i]
        
        # Crop the object from the image
        crop_img = img.crop((int(box[0]), int(box[1]), int(box[0])+int(box[2]), int(box[1])+int(box[3])))
        
        # Paste the cropped object onto the new image
        new_img.paste(crop_img, (int(box[0]), int(box[1]), int(box[0])+int(box[2]), int(box[1])+int(box[3])))

    # Save the new image
    if len(results[k].boxes) == 0:
        test_data.append(np.asarray(imgs[k]))
    else:    
        test_data.append(np.asarray(new_img))


print(len(test_data))    


In [ ]:
import keras

train_data = np.array(train_data)
test_data = np.array(test_data)

train_data = train_data.reshape(train_data.shape[0], 256, 256, 1).astype('float32')
test_data = test_data.reshape(test_data.shape[0], 256, 256, 1).astype('float32')

train_data /= 255.0
test_data /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)



In [ ]:

import matplotlib.pyplot as plt
from random import randint
%matplotlib inline  
idx = randint(0, len(train_data))
# while int(y_train[idx])!= 2: 
idx = randint(0, len(train_data))

plt.imshow(train_data[idx].reshape(256,256)) 
print((y_train[idx]))
print("idx: ", idx)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))  
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())


model.fit(x = train_data, y = y_train, validation_split=0.1, batch_size = 128, verbose=1, epochs=5)

In [ ]:
score = model.evaluate(test_data, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

In [ ]:
model.save('ImageAbstractionModel.h5')